## Дашборд для телекоммуникационной компании

Цель проекта- определить текущий уровень потребительской лояльности (NPS) среди клиентов из России.

Скачиваем файл с базой данных.

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/.../...._csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

***Описание данных таблицы location - содержит справочник территорий, в которых телеком-компания оказывает услуги.***


- location_id -	Идентификатор записи, первичный ключ
- country -	Страна
- city -	Город


In [3]:
query = """
SELECT * FROM location;
"""

In [4]:
df_1 = pd.read_sql(query, engine)
df_1.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


***Описание данных таблицы age_segment - содержит данные о возрастных сегментах клиентов.***


- age_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента


In [5]:
query = """
SELECT *
FROM age_segment;
"""

In [6]:
df_2= pd.read_sql(query, engine)
df_2.head(3)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34


***Описание данных таблицы traffic_segment - содержит данные о выделяемых сегментах по объёму потребляемого трафика.***


- tr_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента


In [7]:
query = """
SELECT *
FROM traffic_segment;
"""

In [8]:
df_3= pd.read_sql(query, engine)
df_3.head(3)

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1


***Описание таблицы lifetime_segment - содержит данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.***


- lt_gr_id -	Идентификатор сегмента, первичный ключ
- bucket_min -	Минимальная граница сегмента
- bucket_max -	Максимальная граница сегмента
- title -	Название сегмента


In [9]:
query = """
SELECT *
FROM lifetime_segment;
"""

In [10]:
df_4= pd.read_sql(query, engine)
df_4.head(3)

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3


***Описание данных таблицы user - содержит основную информацию о клиентах***

- user_id -	Идентификатор клиента, первичный ключ таблицы
- lt_day -	Количество дней «жизни» клиента
- age -	Возраст клиента в годах
- gender_segment -	Пол клиента (1 – женщина, 0 – мужчина)
- os_name -	Тип операционной системы
- cpe_type_name -	Тип устройства
- location_id -	Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id -	Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id -	Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- lt_gr_id -	Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score -	Оценка клиента в NPS-опросе (от 1 до 10)

In [11]:
query = """
SELECT *
FROM user;
"""

In [12]:
df_5= pd.read_sql(query, engine)
df_5.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


Преобразовываем данные с помощью SQL-запроса, чтобы собрать в одну витрину данные из разных таблиц.


***Описание базы данных для дашборда:***

- user_id -	Идентификатор клиента
- lt_day -	Количество дней «жизни» клиента
- is_new -	Поле хранит информацию о том, является ли клиент новым
- age -	Возраст
- gender_segment -	Пол (для удобства работы с полем преобразуйте значения в текстовый вид)
- os_name -	Тип операционной системы
- cpe_type_name -	Тип устройства
- country -	Страна проживания
- city -	Город проживания
- age_segment -	Возрастной сегмент
- traffic_segment -	Сегмент по объёму потребляемого трафика
- lifetime_segment -	Сегмент по количеству дней «жизни»
- nps_score -	Оценка клиента в NPS-опросе
- nps_group -	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе


In [13]:
query = """
SELECT 
    u.user_id,
    u.lt_day,
    CASE
        WHEN lt_day <= 365 THEN 'new'
        ELSE 'old' END
        AS is_new,
    u.age,
    CASE 
        WHEN u.gender_segment = 1 THEN 'female'
        WHEN u.gender_segment = 0 THEN 'male'
        ELSE 'unknown' END
        AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    a.title AS age_segment,
    ts.title AS traffic_segment,
    ls.title AS lifetime_segment,
    u.nps_score,
    CASE
        WHEN nps_score >= 9 THEN 'promoters'
        WHEN nps_score <= 6 THEN 'detractors'
        ELSE 'passives' END
        AS nps_group
FROM
    "user" u 
LEFT JOIN
    location l 
    ON u.location_id = l.location_id
LEFT JOIN
    age_segment a
    ON u.age_gr_id = a.age_gr_id
LEFT JOIN
    traffic_segment ts 
    ON u.tr_gr_id = ts.tr_gr_id 
LEFT JOIN
    lifetime_segment ls 
    ON u.lt_gr_id = ls.lt_gr_id
WHERE CAST(lt_day AS INTEGER) > 0;
"""

In [14]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters


Получившуюся таблицу сохраняем как .csv файл.

In [15]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/marina7800/viz/Dashboard_NPS/Dashboard1#1

### Укажите ссылку на pdf-файл с презентацией:

https://disk.yandex.ru/i/WbTwbuCNcS32Yg